<a href="https://colab.research.google.com/github/ar851060/ESUN_AI_2021_winter/blob/main/5_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import pathlib

In [ ]:
def generate_sample(tag, amt):
  comb = []
  amt = amt[amt['shop_tag']==tag]
  for i in range(12,25):
    data = pd.read_csv("/content/drive/Shareddrives/統學期末比賽/Clear_data_1214/data_"+str(i)+".csv")
    data = data[data['shop_tag']==tag]
    # 丟掉slam、gender_code、primary_card
    data = data.drop(['Unnamed: 0','poscd','overseas_offline_cnt','overseas_offline_amt_pct','slam', 'primary_card', 'masts', 'naty','cuorg'], axis=1)

    # 合併txn_cnt, 只留1,2,4,6
    others = ["card_"+str(x)+"_txn_cnt" for x in [3,5,7,8,9,10,11,12,13,14,"other"]]
    data['card_others_txn_cnt'] = data[others].sum(axis=1)
    data.drop(others, axis=1, inplace = True)

    others = ["card_"+str(x)+"_txn_amt_pct" for x in range(1,15)]
    # data['card_others_txn_amt_pct'] = data[others].sum(axis=1)
    data.drop(others, axis=1, inplace = True)
    data.drop("card_other_txn_amt_pct", axis = 1, inplace = True)

    comb.append(data)
  data = pd.concat(comb)
  data = data.drop(["txn_amt","shop_tag"], axis = 1)
  amt = amt.drop("shop_tag", axis = 1)
  data = data.merge(amt,how="left",on=["dt","chid"])
  return data

In [ ]:
# 類別資料encoding
def encoding(data):
  # data.naty = data.naty.map({1:1,2:0})
  # data_dum = pd.get_dummies(data.masts, prefix="masts")
  # data = data.drop(['masts'],axis = 1)
  # data = pd.concat([data, data_dum], axis = 1)
  # del data_dum
  df_frequency_map = data.trdtp.value_counts().to_dict()
  data.trdtp = data.trdtp.map(df_frequency_map)
  # df_frequency_map = data.cuorg.value_counts().to_dict()
  # data.cuorg = data.cuorg.map(df_frequency_map)
  del df_frequency_map
  return data

In [ ]:
# 產生下個月的目標
def generate_target(data):
  data['last_dt'] = data['dt']-1
  comb = []
  for i in range(12,25):
    now = data[data['dt']==i]
    fut = data[data['dt']==i+1][["last_dt","chid","shop_tag","txn_amt"]]
    now = pd.merge(now, fut, how = "left", left_on = ["dt","chid","shop_tag"], right_on = ["last_dt","chid","shop_tag"]).fillna(0)
    comb.append(now)
  data = pd.concat(comb)
  del now
  del fut
  data = data.drop(["last_dt_x","last_dt_y"],axis = 1)
  data.columns = data.columns.str.replace('txn_amt_y', 'next_txn_amt')
  data.columns = data.columns.str.replace('txn_amt_x', 'txn_amt')
  return data

In [ ]:
# 增加 count
def generate_count(data, tag):
  cou = pd.read_csv("/content/drive/Shareddrives/統學期末比賽/feature/count_before.csv")
  cou = cou[cou['shop_tag']==int(tag)].drop("shop_tag",axis = 1)
  cou = pd.melt(cou, id_vars = "chid").astype(int)
  cou.columns = ["chid", "dt", "count"]
  data = data.merge(cou, how = "left", on = ["chid", "dt"]).fillna(0)
  del cou
  return data

In [ ]:
# 增加 total
def generate_total(data, tag):
  cou = pd.read_csv("/content/drive/Shareddrives/統學期末比賽/feature/total_before.csv")
  cou = cou[cou['shop_tag']==int(tag)].drop("shop_tag",axis = 1)
  cou = pd.melt(cou, id_vars = "chid").astype(float)
  cou.columns = ["chid", "dt", "total"]
  data = data.merge(cou, how = "left", on = ["chid", "dt"]).fillna(0)
  del cou
  return data

In [ ]:
# 提供baseline的答案
def baseline(data,tag):
  base = pd.read_csv("/content/drive/Shareddrives/統學期末比賽/Data/base_answer_12months.csv")
  base = pd.melt(base, id_vars = 'chid')
  base = base[base['value']==int(tag)]
  base['base_ans'] = np.where(base['variable']=="top1", 3,0)
  base['base_ans'] = np.where(base['variable']=="top2", 2,0)
  base['base_ans'] = np.where(base['variable']=="top3", 1,0)
  base = base.drop(["variable","value"],axis = 1)
  data = data.merge(base, how = "left", on = 'chid').fillna(0)
  return data

In [ ]:
# 增加 OtherLift
def generate_lift(data, tag):
  lift = pd.read_csv("/content/drive/Shareddrives/統學期末比賽/feature/other_lift.csv")
  lift = lift.reset_index()
  lift = lift[["chid","dt",tag]]
  lift.columns = ["chid", "dt", "OtherLift"]
  data = data.merge(lift, how = "left", on = ["chid", "dt"]).fillna(0)
  del lift
  return data

In [ ]:
# 製造資料
def make_test(data):
  cust = set(pd.read_csv("/content/drive/Shareddrives/統學期末比賽/Data/target.csv")['chid'].to_list())
  temp = data.loc[data.groupby('chid').dt.idxmax()].reset_index(drop=True)
  cust = cust - set(temp.chid.to_list())
  del temp
  for i in range(24,11,-1):
    if len(cust) == 0: break
    df = pd.read_csv("/content/drive/Shareddrives/統學期末比賽/Clear_data_1214/data_"+str(i)+".csv")
    df = df[df['chid'].isin(cust)]
    # 丟掉slam、gender_code、primary_card
    df = df.drop(['Unnamed: 0','poscd','overseas_offline_cnt','overseas_offline_amt_pct','naty','cuorg','masts','slam', 'primary_card', 'cuorg'], axis=1)

    # 合併txn_cnt, 只留1,2,4,6
    others = ["card_"+str(x)+"_txn_cnt" for x in [3,5,6,7,8,9,10,11,12,13,14,"other"]]
    df['card_others_txn_cnt'] = df[others].sum(axis=1)
    df.drop(others, axis=1, inplace = True)

    others = ["card_"+str(x)+"_txn_amt_pct" for x in range(1,15)]
    # df['card_others_txn_amt_pct'] = df[others].sum(axis=1)
    df.drop(others, axis=1, inplace = True)
    df.drop("card_other_txn_amt_pct", axis=1, inplace = True)

    df[['txn_cnt', 'txn_amt', 'domestic_offline_cnt',
        'domestic_online_cnt', 'overseas_online_cnt',
        'domestic_offline_amt_pct', 'domestic_online_amt_pct',
        'overseas_online_amt_pct', 'card_1_txn_cnt',
        'card_2_txn_cnt', 'card_4_txn_cnt', 'card_others_txn_cnt']] = 0
    # df[['txn_cnt', 'txn_amt']] = 0
    df['shop_tag'] = tag

    cust = cust - set(df.chid.to_list())
    data = pd.concat([data, df], ignore_index = True)
    del df
  return data

In [ ]:
# 對txn_amt做log
def log_txn_amt():
  comb = []
  target_col = ["chid","dt","shop_tag","txn_amt"]
  for i in range(12,25):
    data = pd.read_csv("/content/drive/Shareddrives/統學期末比賽/Data/date_"+str(i)+".csv")
    data = data[target_col]
    comb.append(data)
  data = pd.concat(comb)
  data["txn_amt"] = np.log(data["txn_amt"])
  # data["txn_amt"] = (data["txn_amt"]-data["txn_amt"].min())/(data["txn_amt"].max()-data["txn_amt"].min())
  return data

In [ ]:
pred = map(str, [2,6,10,12,13,15,18,19,21,22,25,26,36,37,39,48])
amt = log_txn_amt()
for tag in pred:
  print("generate sample...")
  data = generate_sample(tag, amt)
  print("generate test...")
  data = make_test(data)
  print("encoding...")
  data = encoding(data)
  print("generate_count...")
  data = generate_count(data, tag)
  print("generate_total...")
  data = generate_total(data, tag)
  print("generate_target...")
  data = generate_target(data)
  print("generate_base...")
  data = baseline(data,tag)
  print("generate_lift...")
  data = generate_lift(data,tag)

  # train test split
  print(data.chid.nunique())
  print("create new folder...")
  path = pathlib.Path("/content/drive/Shareddrives/統學期末比賽/train_test/"+tag)
  path.mkdir(parents=True, exist_ok=True)
  print("saving csv...")
  df = data[data['dt']<24]
  temp = data.loc[data.groupby('chid').dt.idxmax()].reset_index(drop=True)
  print(temp.chid.nunique())
  temp[['chid','shop_tag']].to_csv("/content/drive/Shareddrives/統學期末比賽/train_test/"+tag+"/test_tag_"+tag+"_rank.csv", index = False)
  data[data['dt']==23].drop(['chid',"next_txn_amt"],axis =1).to_csv("/content/drive/Shareddrives/統學期末比賽/train_test/"+tag+"/valx_"+tag+"_rank.csv", index = False)
  data[data['dt']==23]['next_txn_amt'].to_csv("/content/drive/Shareddrives/統學期末比賽/train_test/"+tag+"/valy_"+tag+"_rank.csv", index = False)
  df.drop(["next_txn_amt"], axis = 1).to_csv("/content/drive/Shareddrives/統學期末比賽/train_test/"+tag+"/trainx_"+tag+"_rank.csv", index = False)
  df['next_txn_amt'].to_csv("/content/drive/Shareddrives/統學期末比賽/train_test/"+tag+"/trainy_"+tag+"_rank.csv", index = False)
  temp.drop(['chid',"next_txn_amt"],axis =1).to_csv("/content/drive/Shareddrives/統學期末比賽/train_test/"+tag+"/testx_"+tag+"_rank.csv", index = False)
  del df
  del temp
  print("success~"+tag)

generate sample...
generate test...
encoding...
generate_count...
generate_total...
generate_target...
generate_base...
generate_lift...
500000
create new folder...
saving csv...
500000
success~2
generate sample...
generate test...
encoding...
generate_count...
generate_total...
generate_target...
generate_base...
generate_lift...
500000
create new folder...
saving csv...
500000
success~6
generate sample...
generate test...
encoding...
generate_count...
generate_total...
generate_target...
generate_base...
generate_lift...
500000
create new folder...
saving csv...
500000
success~10
generate sample...
generate test...
encoding...
generate_count...
generate_total...
generate_target...
generate_base...
generate_lift...
500000
create new folder...
saving csv...
500000
success~12
generate sample...
generate test...
encoding...
generate_count...
generate_total...
generate_target...
generate_base...
generate_lift...
500000
create new folder...
saving csv...
500000
success~13
generate sample..

Boxcox transformation

In [ ]:
from scipy.stats import boxcox

In [ ]:
comb = []
target_col = ["txn_amt"]
for i in range(1,25):
  data = pd.read_csv("/content/drive/Shareddrives/統學期末比賽/Data/date_"+str(i)+".csv")
  data = data[target_col]
  comb.append(data)
data = pd.concat(comb)


In [ ]:
_, lam = boxcox(data.T.to_numpy()[0])

/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:205: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)


In [ ]:
lam

-0.013085205871723776